In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    # fit(), predict() method만 재정의 (override)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0]) # X_test의 179개의 값을 전부 0으로 대입
        for i in range(X.shape[0]):
            if X[i, 1] == 0:        # 여성이면 1로 바꾸겠다
                pred[i] = 1
        
        return pred

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [4]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
df = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'deck']]
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,NaN


In [7]:
# age 컬럼은 평균으로 대체

df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [8]:
# embarked는 최빈값으로 대체

df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [9]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [10]:
# deck 컬럼은 삭제
df.drop(columns = ['deck'], inplace=True)

* 카테고리 값인 sex, embarked 컬럼은 숫자로 변환

In [11]:
# LabelEncoder로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [12]:
df.sex = le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


### 2. Train/Test dataset으로 분리

In [13]:
# X와 y를 넘파이 배열로 (values 안 붙이면 DataFrame이 된다)
# 데이터 프레임으로 할 경우 나중에 X_test 데이터 뽑고 싶을 때 X_test[0]이 아니라 X_test.iloc[0]으로 주어야 한다.

X = df.iloc[:, 1:].values
y = df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=11
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [16]:
def get_clf_eval(y_test, pred=None, ):    # pred의 default를 None으로 주겠다
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    print('오차 행렬')
    print(confusion)
    print(f'정확도 : {accuracy:.4f}, 정밀도 : {precision:.4f}, 재현율 : {recall:.4f}', end = '')

In [17]:
lr = LogisticRegression(max_iter = 500)

lr.fit(X_train, y_train)
pred = lr.predict(X_test)    # X_test에 대한 예측, pred 정의
get_clf_eval(y_test, pred)   # 함수 호출

오차 행렬
[[97 13]
 [19 50]]
정확도 : 0.8212, 정밀도 : 0.7937, 재현율 : 0.7246


In [23]:
pred = lr.predict(X_test)
pred


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [18]:
# 정밀도가 세로 ( TP / TP + FP )  -> type1 에러와 관련
# 재현율은 가로 ( TP / FN + TP )  -> type2 에러와 관련